# Week of 2025.2.23

Last week we split the domain in half. Half of the domain (-1000, -500) is the liquid melt which viscous frictional force is applied to. The other half of the domain is the plug which shear frictional force is applied. The crystal content is the only difference in the makeup between the two regions. In the plug portion of domain, the crystal partial density is 50 while the melt currently has no crystals.  

To verify the numerical solution lets consider the analytical solution

## 1.0 Analytical solution 

Let's first consider the analytical problem. First the force balance for the plug can be written like so

$$
M \ddot{s} = A (p0 + \Delta p(s(t))) - P_{atm} A - 2 \pi R L \tau(s(t))
$$

We are basically just stating that the force exerted on the plug is the delta in pressure multiplied by the cross section surface area, subtracted by the shear stress multiplied by the surface area of the conduit. 

Equilibrium is defined as 

$$
\begin{align}
0 = P_0 A - P_{atm}A - 2 \pi R L \tau(0)\\
P_0 = p_{atm} + \frac{2 \pi R L \tau_p}{A} \\ 
P_0 = p_{atm} + \frac{2 L \tau_p}{R}
\end{align}
$$

using that condition above, we get a stability equation of:

$$
\begin{align}
M \ddot{s} = A \Delta p(s(t)) + 2 \pi R L (\tau_p - \tau(s)) \\
M \ddot{s} = \frac{-A^2s(t)}{S} + 2 \pi R L (\tau_p - \tau(s))
\end{align}
$$

Where $S$ is known as storage capacity. This is acceleration. So whenever this quantity is > 0 the system is unstable. 

As a first step, lets set the pressure gradient such that the system stays in equilibrium. Because the system will be in equilibrium, there will be no velocity in the system and the viscous drag term aught to be zero. As a result, we expect to see a pressure gradient across the plug starting low at $p_{atm} = 1e5$ [Pa] and increasing to $P_{-500}$. To calculate that we solve the following:

$$
P_{-500} = p_{atm} + \frac{2 L \tau_p}{R} \\
P_{-500} = 1e5 [Pa] + \frac{2 * 500 [m] * 2e5 [Pa]}{50 * [m]} \\ 
P_{-500} = 4.1e6 [Pa]
$$

To verify the steady state I am going to run the simulation with the $P_r = 1e5$ and $P_l = 4.1e6$. Based on our calculations that should result in a static solution.



## 1. 1 Python Imports

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Modify base path for depending on your file structure.
BASE_PATH = "/Users/paxton/git"

# Specify path where .pkl files are located
target_dir = f"{BASE_PATH}/quail_volcano/scenarios/simple_1D_test"
# Specify path for Quail source code
source_dir = f"{BASE_PATH}/quail_volcano/src"
# Change to working directory
os.chdir(target_dir)


# Import quail modules
os.chdir(source_dir)

import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing.animate as animate

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
from IPython.display import HTML
import multiprocessing as mp  
from multidomain import Domain, Observer

os.chdir(target_dir)

## 1.2 Numerical static, steady state solution

* I used a SlipWall boundary condition on the left side. 
* The plug/melt boundary is defined as the point at which the crystal partial density = 25. That threshold is graphed below.
* The slip is basically zero, but I does go slightly negative at times as a result of the pressure gradient in the left half of the domain having a slightly positive slope at times.
* Per Eric's suggestion, I also reduced the water content down to a partial density of 0.00001. 

In [4]:
ani = animate.animate_conduit_pressure("slip_variable_addition", iterations=100, file_prefix="split_domain_steady_state")

HTML(ani.to_html5_video())

/Users/paxton/git/quail_volcano/scenarios/simple_1D_test


## 1.3 Dynamic, higher pressure than steady state 

Now lets increase $P_L = 10e6 [Pa]$ and verify that the solution is no longer static. 

Let's review compresability:

$$
\begin{align}
K &= - V \frac{d P}{d V} \\

\Delta V &= - \frac{V}{K} (\Delta P) \\

\Delta V &= - \frac{V}{K} (\Delta P) \\
\end{align}
$$

Note that:
$$
\begin{align}
\Delta V &= \Delta L * A \\
V &= L * A \\
\end{align}
$$
So:

$$
\begin{align}
\Delta L &= - \frac{L}{K} \Delta P \\
\Delta L &= - \frac{500 [m]}{10e9 [Pa]} (-6e6) [Pa] \\
\Delta L &= 0.3
\end{align}
$$


So, based on our initial pressure we expect to see a slip of 0.3 meters. We currently see a slip of about 0.17m. 

I believe the error is result of velocity being negative. When I clip the velocity (which is a source term for slip), the slip in this problem is about 0.6m. 



In [5]:
ani = animate.animate_conduit_pressure("slip_variable_addition", iterations=100, file_prefix="split_domain_dynamic")

HTML(ani.to_html5_video())

/Users/paxton/git/quail_volcano/scenarios/simple_1D_test


## 1.4 Decrease $D_c$ to 0.1

Let us verify that when we reduce $D_c$ to 0.1 we see the shear stress decrease as the plug slips further. Note the current values are:

1. $\tau_p = 2e5$
2. $\tau_r = 5e4$

Because the ultimate $\tau$ is now lower, we can calculate again the expected steady state pressure on the left of the plug.

$$
P_{-500} = p_{atm} + \frac{2 L \tau_p}{R} \\
P_{-500} = 1e5 [Pa] + \frac{2 * 500 [m] * 5e4 [Pa]}{50 * [m]} \\ 
P_{-500} = 1.1e6 [Pa]
$$

That analytical estimate is correctly verified below


In [24]:
ani = animate.animate_conduit_pressure("slip_variable_addition", iterations=100, file_prefix="split_domain_small_dc")

HTML(ani.to_html5_video())

/Users/paxton/git/quail_volcano/scenarios/simple_1D_test


## 1.5 $D_c = 0.1$ and $\tau_r = 0$

Let's now experiment with the case where we reduce the residual shear stress to zero. As a result, the plug should be fully ejected until the $P_R$ matches the atmosphere pressure $1e5 [Pa]$.



In [2]:
ani = animate.animate_conduit_pressure("slip_variable_addition", iterations=100, file_prefix="split_domain_tau_r_0")

HTML(ani.to_html5_video())

/Users/paxton/git/quail_volcano/scenarios/simple_1D_test


## Open questions
1. How do we solve the issue of negative velocity (and therefore a negative source term for slip)? Better to make source term absolute value? 
